<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [1]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.21.5
Pandas:  1.4.2


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [3]:
# wrong:
bikes = pd.read_table('/content/bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('/content/bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('/content/bikeshare.csv', header = 0)
print(bikes.head())

FileNotFoundError: [Errno 2] No such file or directory: '/content/bikeshare.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [13]:
#ANSWER:
bikes = pd.read_table("../DATA/bikeshare.csv", header = 0, sep = ',')
print(bikes.head())

   instant      dteday  season  yr  mnth  hr  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1   0        0        6           0   
1        2  2011-01-01       1   0     1   1        0        6           0   
2        3  2011-01-01       1   0     1   2        0        6           0   
3        4  2011-01-01       1   0     1   3        0        6           0   
4        5  2011-01-01       1   0     1   4        0        6           0   

   weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
0           1  0.24  0.2879  0.81        0.0       3          13   16  
1           1  0.22  0.2727  0.80        0.0       8          32   40  
2           1  0.22  0.2727  0.80        0.0       5          27   32  
3           1  0.24  0.2879  0.75        0.0       3          10   13  
4           1  0.24  0.2879  0.75        0.0       0           1    1  


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [7]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [15]:
df = pd.read_excel("../DATA/Iris.xls", sheet_name = 'Data')
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [16]:
#ANSWER
df = pd.read_excel('../DATA/Iris.xls', sheet_name = 'Data', header = 0, usecols = [0, 1, 2, 3, 4])
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length
0,1,0.2,1.4,3.5,5.1
1,1,0.2,1.4,3.0,4.9
2,1,0.2,1.3,3.2,4.7
3,1,0.2,1.5,3.1,4.6
4,1,0.2,1.4,3.6,5.0
...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7
146,3,1.9,5.0,2.5,6.3
147,3,2.0,5.2,3.0,6.5
148,3,2.3,5.4,3.4,6.2


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [17]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-11-24T02:11:24.414Z,-4.3351,100.4133,9.11,5.1,mb,64,98,2.535,0.71,...,2022-11-24T02:28:00.040Z,"southwest of Sumatra, Indonesia",earthquake,7.09,4.64,0.077,54,reviewed,us,us


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [18]:
url = 'https://www.ccra.com/airport-codes/'
df = pd.read_html(url)
df

[                City      Country Code
 0            Aalborg      Denmark  AAL
 1           Aalesund       Norway  AES
 2             Aarhus      Denmark  AAR
 3     Abbotsford, BC       Canada  YXX
 4     Abbotsford, BC       Canada  YXX
 ...              ...          ...  ...
 1574        Zanzibar     Tanzania  ZNZ
 1575        Zaragoza        Spain  ZAZ
 1576       Zhengzhou        China  CGO
 1577        Zhoushan        China  HSN
 1578          Zurich  Switzerland  ZRH
 
 [1579 rows x 3 columns]]

In [19]:
df[0]

,City,Country,Code
0,Aalborg,Denmark,AAL
1,Aalesund,Norway,AES
2,Aarhus,Denmark,AAR
3,"Abbotsford, BC",Canada,YXX
4,"Abbotsford, BC",Canada,YXX
...,...,...,...
1574,Zanzibar,Tanzania,ZNZ
1575,Zaragoza,Spain,ZAZ
1576,Zhengzhou,China,CGO
1577,Zhoushan,China,HSN


#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [20]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [22]:
#ANSWER
bikes.shape

(17379, 17)

What are the column names in `bikes`?

In [23]:
#ANSWER
bikes.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

What are the data types of these columns?

In [27]:
#ANSWER
bikes.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [28]:
#ANSWER
bikes.index

RangeIndex(start=0, stop=17379, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [32]:
#ANSWER
print(bikes['season'].head())
print(type(bikes['season']))
print(bikes[['season']].head())
print(type(bikes[['season']]))

0    1
1    1
2    1
3    1
4    1
Name: season, dtype: int64
<class 'pandas.core.series.Series'>
   season
0       1
1       1
2       1
3       1
4       1
<class 'pandas.core.frame.DataFrame'>


How would we use object notation to show the first 4 rows of `atemp`?

In [34]:
#ANSWER
print(bikes.atemp[:4])

0    0.2879
1    0.2727
2    0.2727
3    0.2879
Name: atemp, dtype: float64


Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [37]:
bikes[[0]]
bikes[0]
bikes[0,0]
bikes[[0,0]]

KeyError: "None of [Int64Index([0], dtype='int64')] are in the [columns]"

What is the correct way to access the 1st row of the DataFrame by its index?

In [38]:
#ANSWER
bikes[:1]

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16


What is the correct way to access the 2nd column of the DataFrame by its index?

In [39]:
#ANSWER
bikes.iloc[:, 1]

0        2011-01-01
1        2011-01-01
2        2011-01-01
3        2011-01-01
4        2011-01-01
            ...    
17374    2012-12-31
17375    2012-12-31
17376    2012-12-31
17377    2012-12-31
17378    2012-12-31
Name: dteday, Length: 17379, dtype: object

## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [ ]:
bikes.isnull().head()

However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [43]:
#ANSWER
bikes['windspeed'].isnull().sum()

0

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [44]:
#ANSWER
bikes.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [45]:
#ANSWER
def divide(divident, divisor):
    if divisor == 0:
        quotient = np.nan
    else:
        quotient = divident / divisor
    return (quotient)

print(divide(1, 0))

nan


Apply the Pandas `isna` function to the following data objects:

In [46]:
x = 2.3
y = np.nan
print(x, y)

2.3 nan


In [47]:
#ANSWER
print(pd.isna(x), pd.isna(y))

False True


In [48]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [49]:
#ANSWER
print(pd.isna(array))

[[False  True False]
 [False False  True]]


How is the pandas I/O parameter `na_values` used?

? ANSWER: 
For converting specially coded strings to NaN

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [50]:
#ANSWER
bikes['season'].value_counts()

3    4496
2    4409
1    4242
4    4232
Name: season, dtype: int64

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [51]:
#ANSWER
print('instant:', bikes['instant'].min(), 'to', bikes['instant'].max())
print('dteday:', bikes['dteday'].min(), 'to', bikes['dteday'].max())
print('windspeed:', bikes['windspeed'].min(), 'to', bikes['windspeed'].max())

instant: 1 to 17379
dteday: 2011-01-01 to 2012-12-31
windspeed: 0.0 to 0.8507


Compute and print the overall minimum and maximum of the numeric data columns:

In [53]:
bikes_min, bikes_max = (min(bikes.min(numeric_only=True)), 
                        max(bikes.max(numeric_only=True)))
bikes_min, bikes_max

(0.0, 17379.0)

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [54]:
bikes['atemp'].quantile(0.5)

0.4848

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [55]:
#ANSWER
bikes['atemp'].quantile((0.1, 0.25, 0.5, 0.75, 0.9))

0.10    0.2424
0.25    0.3333
0.50    0.4848
0.75    0.6212
0.90    0.6970
Name: atemp, dtype: float64

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [56]:
type(bikes['atemp'])

pandas.core.series.Series

In [57]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
2773,2774,2011-04-30,2,0,4,11,0,6,0,1,0.50,0.4848,0.42,0.3582,120,201,321
3210,3211,2011-05-18,2,0,5,16,0,3,1,1,0.60,0.5909,0.73,0.1940,53,241,294
8174,8175,2011-12-12,4,0,12,6,0,1,1,1,0.16,0.1818,0.80,0.1045,1,63,64
12922,12923,2012-06-27,3,1,6,15,0,3,1,1,0.80,0.6970,0.26,0.4627,61,188,249
13326,13327,2012-07-14,3,1,7,11,0,6,0,2,0.72,0.6818,0.70,0.1045,143,296,439


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [58]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)         

What is `atemp_level`?

In [59]:
#ANSWER
print(type(atemp_level))

<class 'pandas.core.series.Series'>


Here is a random sample of `atemp_level`:

In [60]:
atemp_level.sample(5)

6254        (0.25, 0.5]
16241       (0.25, 0.5]
4289        (0.5, 0.75]
7359        (0.25, 0.5]
17330    (-0.001, 0.25]
Name: atemp, dtype: category
Categories (4, interval[float64, right]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [61]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

16578    mild
15444    warm
878      cool
12186    warm
7673     mild
Name: atemp, dtype: category
Categories (4, object): ['cool' < 'mild' < 'warm' < 'hot']

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [62]:
#ANSWER
bikes['atemp_level'] = atemp_level
bikes[(bikes.atemp_level == 'mild') & (bikes.season == 2)].season.count()

1829

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [63]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [ ]:
# Cleaning Data

## Load Data

Load rock.csv and clean the dataset.

In [64]:
rock_csv = '../DATA/rock.csv'
rock = pd.read_csv(rock_csv)

## Check Column Names

Check column names and clean.

In [65]:
rock.columns

Index(['Song Clean', 'ARTIST CLEAN', 'Release Year', 'COMBINED', 'First?',
       'Year?', 'PlayCount', 'F*G'],
      dtype='object')

In [67]:
def clean_column_name(column_names):
    clean_column_names = []
    for c in column_names:
        c = c.lower().replace(' ', '_')
        c = c.lower().replace('*', '')
        c = c.lower().replace('?', '')
        clean_column_names.append(c)
        
    return clean_column_names

In [68]:
rock.columns = clean_column_name(rock.columns)

In [69]:
rock.columns

Index(['song_clean', 'artist_clean', 'release_year', 'combined', 'first',
       'year', 'playcount', 'fg'],
      dtype='object')

## Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [70]:
rock.isnull().sum()

song_clean        0
artist_clean      0
release_year    577
combined          0
first             0
year              0
playcount         0
fg                0
dtype: int64

In [71]:
null_mask = rock['release_year'].isnull()
rock.loc[null_mask, 'release_year'] = 0

In [72]:
rock.isnull().sum()

song_clean      0
artist_clean    0
release_year    0
combined        0
first           0
year            0
playcount       0
fg              0
dtype: int64

## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [73]:
rock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   song_clean    2230 non-null   object
 1   artist_clean  2230 non-null   object
 2   release_year  2230 non-null   object
 3   combined      2230 non-null   object
 4   first         2230 non-null   int64 
 5   year          2230 non-null   int64 
 6   playcount     2230 non-null   int64 
 7   fg            2230 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 139.5+ KB


In [74]:
rock['release_year'].value_counts()

0                577
1973             104
1977              83
1975              83
1970              81
1971              75
1969              72
1980              70
1978              64
1979              63
1981              61
1967              61
1983              60
1976              56
1982              54
1984              51
1972              50
1974              48
1968              46
1985              39
1987              39
1986              37
1991              34
1989              32
1966              30
1988              29
1965              28
1994              25
1990              22
1993              19
1964              14
1992              14
1999              13
1995              10
1997               9
1963               9
1996               9
1998               6
2002               6
2005               5
2004               5
2012               5
2001               4
2007               3
1962               3
2000               3
2011               3
2003         

In [75]:
song_facts_mask = rock['release_year'] == 'SONGFACTS.COM'
rock.loc[song_facts_mask, 'release_year'] = np.NaN                    

In [95]:
rock['release_year'] = pd.to_numeric(rock['release_year'])

In [78]:
rock.dtypes

song_clean       object
artist_clean     object
release_year    float64
combined         object
first             int64
year              int64
playcount         int64
fg                int64
dtype: object

## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [79]:
def check_min_max(df):
    print(df.describe().T[['min', 'max']])

In [80]:
check_min_max(rock)

              min     max
release_year  0.0  2014.0
first         1.0     1.0
year          0.0     1.0
playcount     0.0   142.0
fg            0.0   142.0


In [81]:
song_facts_mask = rock['release_year'] == 0
rock.loc[song_facts_mask, 'release_year'] = np.NaN

In [82]:
check_min_max(rock)

                 min     max
release_year  1071.0  2014.0
first            1.0     1.0
year             0.0     1.0
playcount        0.0   142.0
fg               0.0   142.0


In [83]:
song_facts_mask = rock['release_year'] == 1071
rock.loc[song_facts_mask, 'release_year'] = np.NaN

In [84]:
check_min_max(rock)

                 min     max
release_year  1955.0  2014.0
first            1.0     1.0
year             0.0     1.0
playcount        0.0   142.0
fg               0.0   142.0


## Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [85]:
rock.dtypes

song_clean       object
artist_clean     object
release_year    float64
combined         object
first             int64
year              int64
playcount         int64
fg                int64
dtype: object

In [86]:
def check_song(row):
    print('Song: ', row['song_clean'])
    print('Artist: ', row['artist_clean'])
    print('Released before 1970: ', row['release_year'] < 1970)
    print('\n')

In [87]:
rock.iloc[0]

song_clean                     Caught Up in You
artist_clean                        .38 Special
release_year                             1982.0
combined        Caught Up in You by .38 Special
first                                         1
year                                          1
playcount                                    82
fg                                           82
Name: 0, dtype: object

In [88]:
check_song(rock.iloc[0])

Song:  Caught Up in You
Artist:  .38 Special
Released before 1970:  False




In [89]:
rock.head(5).apply(check_song, axis = 1)

Song:  Caught Up in You
Artist:  .38 Special
Released before 1970:  False


Song:  Fantasy Girl
Artist:  .38 Special
Released before 1970:  False


Song:  Hold On Loosely
Artist:  .38 Special
Released before 1970:  False


Song:  Rockin' Into the Night
Artist:  .38 Special
Released before 1970:  False


Song:  Art For Arts Sake
Artist:  10cc
Released before 1970:  False




0    None
1    None
2    None
3    None
4    None
dtype: object

## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [90]:
def convert_to_float(column):
    column = pd.to_numeric(column, errors='coerce')
    return column

## Apply these functions to your dataset

In [91]:
rock.apply(convert_to_float)

,song_clean,artist_clean,release_year,combined,first,year,playcount,fg
0,NaN,NaN,1982.0,NaN,1,1,82,82
1,NaN,NaN,NaN,NaN,1,0,3,0
2,NaN,NaN,1981.0,NaN,1,1,85,85
3,NaN,NaN,1980.0,NaN,1,1,18,18
4,NaN,NaN,1975.0,NaN,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,NaN,NaN,NaN,NaN,1,0,1,0
2226,NaN,NaN,1981.0,NaN,1,1,32,32
2227,NaN,NaN,1975.0,NaN,1,1,109,109
2228,NaN,NaN,1983.0,NaN,1,1,1,1


## Describe the new float-only DataFrame.

In [92]:
rock.describe().T

,count,mean,std,min,25%,50%,75%,max
release_year,1651.0,1978.569352,9.309780,1955.0,1971.0,1977.0,1984.0,2014.0
first,2230.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
year,2230.0,0.741256,0.438043,0.0,0.0,1.0,1.0,1.0
playcount,2230.0,16.872646,25.302972,0.0,1.0,4.0,21.0,142.0
fg,2230.0,15.048430,25.288366,0.0,0.0,3.0,18.0,142.0


>




---



---



> > > > > > > > > © 2022 Institute of Data


---



---



